In [ ]:
import shapefile
import math
import os
import requests
from collections import defaultdict
from tqdm import tqdm
import hashlib
import numpy as np
import json

def deg2num(lat_deg, lon_deg, zoom):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad))) / math.pi) / 2.0 * n)
    return (xtile, ytile)

def deg2num(lat_deg, lon_deg, zoom):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad))) / math.pi) / 2.0 * n)
    return (xtile, ytile)

def num2deg(xtile, ytile, zoom):
    n = 2.0 ** zoom
    lon_deg = xtile / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
    lat_deg = math.degrees(lat_rad)
    return (lat_deg, lon_deg)

def get_work_queue(lats, lons, zoom):
    top_left = deg2num(max(lats), min(lons), zoom)
    bottom_right = deg2num(min(lats), max(lons), zoom)

    tiles = []
    for i in range(top_left[0], bottom_right[0]):
        for j in range(top_left[1], bottom_right[1]):
            tiles.append((i, j + 1))
    return tiles

## Way segments may be spread over multiple tiles, this class helps with merging them

In [ ]:
class ListElement:

    def __init__(self, uri):
        self.uri = uri
        self.prev = None
        self.next = None

    def set_next(self, other_element):
        self.next = other_element

    def set_previous(self, other_element):
        self.prev = other_element


class ListProxy:

    def __init__(self):
        self.elements = {}

    def add_segment(self, raw_elements):
        if raw_elements[0] not in self.elements:
            uri = raw_elements[0]
            new_element = ListElement(uri)
            self.elements[uri] = new_element

        previous = self.elements[raw_elements[0]]

        for raw_element in raw_elements[1:-1]:

            if raw_element not in self.elements:
                new_element = ListElement(raw_element)
                self.elements[raw_element] = new_element
            else:
                new_element = self.elements[raw_element]
            new_element.set_previous(previous)

            previous.set_next(new_element)
            previous = new_element

        if raw_elements[-1] not in self.elements:
            uri = raw_elements[-1]
            new_element = ListElement(uri)
            self.elements[uri] = new_element

        last = self.elements[raw_elements[-1]]
        previous.set_next(last)
        last.set_previous(previous)

    def get_elements(self):
        done = set()
        element = list(self.elements.values())[0]
        i = 0
        while element.prev != None and i < 1000:
            element = element.prev
            i += 1

        yield element
        done.add(element)
        while element.next != None:
            element = element.next
            yield element
            if element in done:
                break
            done.add(element)

## The actual merging code

In [ ]:
def merge_tiles(files, outfile):
    objs = []
    for filename in files:
        with open(filename) as f:
            objs.append(json.load(f))
    with open(outfile, 'w') as f:
        json.dump(merge_tile_data(objs), f)


def merge_tile_data(tiles):
    nodes = {}
    ways = {}
    way_segments = defaultdict(list)

    for tile in tiles:
        (partial_nodes, partial_ways) = tile

        nodes.update(partial_nodes)
        ways.update(partial_ways)

        for way_definition in partial_ways.values():
            way_segments[way_definition['@id'
                         ]].append(way_definition['osm:hasNodes'])

    for way in ways.values():
        node_list = ListProxy()
        for way_segment in way_segments[way['@id']]:
            node_list.add_segment(way_segment)
        node_list = list(n.uri for n in node_list.get_elements())
        way['osm:hasNodes'] = node_list
    return (nodes, ways)

## IO code

In [ ]:
def get_tile(tile_x, tile_y, zoom):
    file_name = get_file_name(tile_x, tile_y, zoom)

    nodes = {}
    ways = {}

    try:
        with open(file_name) as f:
            obj = json.load(f)
            graph = obj['@graph']
            for element in graph:
                if element.get('geo:long'):
                    nodes[element['@id']] = element
                if element.get('osm:hasNodes'):
                    ways[element['@id']] = element
    except Exception as e:
        print(e)

    return (nodes, ways)


def create_tile(tile_x, tile_y, zoom):
    if zoom > 14:
        return ({}, {})

    file_name = get_merged_file_name(tile_x, tile_y, zoom)
    if True or not os.path.isfile(file_name):
        result = []
        nested_tiles = []
        for i in (0, 1):
            for j in (0, 1):
                nested_tiles.append(get_tile(2 * tile_x + i, 2 * tile_y
                                    + j, zoom + 1))
        (merged_nodes, merged_ways) = merge_tile_data(nested_tiles)
        write_to_file(zoom, tile_x, tile_y, merged_nodes, merged_ways)


def write_to_file(
    zoom,
    tile_x,
    tile_y,
    nodes,
    ways,
    ):
    file_name = get_merged_file_name(tile_x, tile_y, zoom)

    graph = list(nodes.values()) + list(ways.values())

    blob = {
        '@context': {
            'tiles': 'https://w3id.org/tree/terms#',
            'hydra': 'http://www.w3.org/ns/hydra/core#',
            'osm': 'https://w3id.org/openstreetmap/terms#',
            'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
            'geo': 'http://www.w3.org/2003/01/geo/wgs84_pos#',
            'dcterms': 'http://purl.org/dc/terms/',
            'dcterms:license': {'@type': '@id'},
            'hydra:variableRepresentation': {'@type': '@id'},
            'hydra:property': {'@type': '@id'},
            'osm:access': {'@type': '@id'},
            'osm:barrier': {'@type': '@id'},
            'osm:bicycle': {'@type': '@id'},
            'osm:construction': {'@type': '@id'},
            'osm:crossing': {'@type': '@id'},
            'osm:cycleway': {'@type': '@id'},
            'osm:footway': {'@type': '@id'},
            'osm:highway': {'@type': '@id'},
            'osm:motor_vehicle': {'@type': '@id'},
            'osm:motorcar': {'@type': '@id'},
            'osm:oneway_bicycle': {'@type': '@id'},
            'osm:oneway': {'@type': '@id'},
            'osm:smoothness': {'@type': '@id'},
            'osm:surface': {'@type': '@id'},
            'osm:tracktype': {'@type': '@id'},
            'osm:vehicle': {'@type': '@id'},
            'osm:hasNodes': {'@container': '@list', '@type': '@id'},
            'osm:hasMembers': {'@container': '@list', '@type': '@id'},
            },
        '@id': 'https://tiles.openplanner.team/planet/{}/{}/{}/'.format(zoom,
                tile_x, tile_y),
        'tiles:zoom': zoom,
        'tiles:longitudeTile': tile_x,
        'tiles:latitudeTile': tile_y,
        'dcterms:isPartOf': {
            '@id': 'https://tiles.openplanner.team/planet/',
            '@type': 'hydra:Collection',
            'dcterms:license': 'http://opendatacommons.org/licenses/odbl/1-0/',
            'dcterms:rights': 'http://www.openstreetmap.org/copyright',
            'hydra:search': {
                '@type': 'hydra:IriTemplate',
                'hydra:template': 'https://tiles.openplanner.team/planet/14/{x}/{y}',
                'hydra:variableRepresentation': 'hydra:BasicRepresentation',
                'hydra:mapping': [{
                    '@type': 'hydra:IriTemplateMapping',
                    'hydra:variable': 'x',
                    'hydra:property': 'tiles:longitudeTile',
                    'hydra:required': True,
                    }, {
                    '@type': 'hydra:IriTemplateMapping',
                    'hydra:variable': 'y',
                    'hydra:property': 'tiles:latitudeTile',
                    'hydra:required': True,
                    }],
                },
            },
        '@graph': graph,
        }

    with open(file_name, 'w') as f:
        json.dump(blob, f)

## Lines of code to tinker with to change the area/zoom level

In [ ]:
def get_file_name(tile_x, tile_y, zoom):
    return 'tiles/tr_{}/{}/{}.json'.format(zoom, tile_x, tile_y)

def get_merged_file_name(tile_x, tile_y, zoom):
    dir_name = 'tiles/{}/{}/'.format(zoom, tile_x)
    if not os.path.isdir(dir_name):
        os.mkdir(dir_name)
    return '{}/{}.json'.format(dir_name, tile_y)

In [ ]:
# bounding box of data that needs to be processed
lats = [49, 52]
lons = [2.25, 6.6]

# zoom level that needs to be created
zoom = 8  

for tile_x, tile_y in tqdm(get_work_queue(lats, lons, zoom)):
    create_tile(tile_x, tile_y, zoom)